# Colab VM configuration

This notebook is intended simply for dumping status information from a Colab runtime (read: kernel).

## Setup system

In [0]:
# Various installs of packages imported by colab_vm_config_info.ipynb
!pip install humanize
!pip install gputil
!pip install psutil

In [0]:
import humanize
import platform
import psutil
import os

In [0]:
# Request TF 2.x, not 1.x
try:
  # %tensorflow_version is a Colab-only thing 
  %tensorflow_version 2.x
except Exception:
  print("TensorFlow 2.x does not seem to be available")

## Python environment


In [0]:
# What version of Python is in effect?
a_message = "Python runtime version: " + platform.python_version() 
print(a_message)

In [0]:
# What packages are installed for the detected running version of Python?
python_major_version = int(platform.python_version_tuple()[0])
print(python_major_version)

if python_major_version == 3:
  print("Python 3.6 dist-packages")
  !ls /usr/local/lib/python3.6/dist-packages
else: 
  # Python 2 it is...
  print("Python 2.7 dist-packages")
  !ls /usr/local/lib/python2.7/dist-packages

There are a LOT of packages pre-installed on Colab.

In [1]:
!pip freeze

absl-py==0.8.1
alabaster==0.7.12
albumentations==0.1.12
altair==3.2.0
astor==0.8.0
astropy==3.0.5
atari-py==0.2.6
atomicwrites==1.3.0
attrs==19.3.0
audioread==2.1.8
autograd==1.3
Babel==2.7.0
backcall==0.1.0
backports.tempfile==1.0
backports.weakref==1.0.post1
beautifulsoup4==4.6.3
bleach==3.1.0
blis==0.2.4
bokeh==1.0.4
boto==2.49.0
boto3==1.10.27
botocore==1.13.27
Bottleneck==1.3.1
branca==0.3.1
bs4==0.0.1
bz2file==0.98
cachetools==3.1.1
certifi==2019.9.11
cffi==1.13.2
chainer==6.5.0
chardet==3.0.4
chart-studio==1.0.0
Click==7.0
cloudpickle==1.2.2
cmake==3.12.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.2.0
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.0
cupy-cuda101==6.5.0
cvxopt==1.2.3
cvxpy==1.0.25
cycler==0.10.0
cymem==2.0.3
Cython==0.29.14
daft==0.0.4
dask==1.1.5
dataclasses==0.7
datascience==0.10.6
decorator==4.4.1
defusedxml==0.6.0
descartes==1.1.0
dill==0.3.1.1
distributed==1.25.3
Django==2.2.7
dlib==19.18.0
dm-sonnet==1.35
docopt==0.6.

## Jupyter


In [0]:
!jupyter-kernelspec list

## CPU

In [0]:
# How many CPUs does this machine have?
!lscpu | grep "^CPU(s):"

In [0]:
# More details on the CPU(s):
!cat /proc/cpuinfo

In [0]:
# RAM
!cat /proc/meminfo | head -n3

In [0]:
# RAM info humanized
process = psutil.Process(os.getpid())
print("RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))


## File system

As with AWS Lambda, there are intentionally few switches for selecting VM options. Memory and CPU are provided as matching packages, not independently configurable.

So, depending on what you ask for in terms of compute (CPU, GPU, [TPU](https://colab.research.google.com/notebooks/tpu.ipynb)) you get more or less file system memory [[*](https://stackoverflow.com/a/55890688)]. Note: for all compute options, the OS files initialize to consuming about 25MB of the file system before you are dropped into the kernel. 

On 2019-05-19, the following tests gave these results:

Processor | FS Free GB | FS Total GB 
--|--|--
CPU | 24 | 49
GPU | 318 | 359
TPU | 26| 49

The low FS size for the TPU is probably because for the TPU case, those (the actually TPU boards) are separate machines while for GPUs those  are a part of the machine the notebook is running on. So, for the CPU and the TPU options, Google is probably providing the same VM, ergo the file systems are essentially the same size. 

In [0]:
!df -h .

## GPU information

First, is a GPU allocated for this notebook's VM to work with?

In [0]:
# What GPU is currently config'd for use?

# https://colab.research.google.com/notebooks/gpu.ipynb
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print("GPU detected: NONE")
else:
  print('GPU detected: {}'.format(device_name))

Assuming we have a GPU, what kind? The answer is in the final line of the following cell's output:

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [0]:
# Memory in GPU
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi

import GPUtil as GPU

gpus = GPU.getGPUs()

if len(gpus) > 0:
  gpu = gpus[0]
  print("GPU RAM:\n Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
else:
  print("GPU detected: NONE")

In [0]:
!nvcc --version
!conda install tsnecuda cuda100 -c cannylab

## References

* [Google Colab Free GPU Tutorial](https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d)
* [GPU stats code](https://stackoverflow.com/questions/48750199/google-colaboratory-misleading-information-about-its-gpu-only-5-ram-available)
* [TensorFlow with GPU](https://colab.research.google.com/notebooks/gpu.ipynb#scrollTo=3IEVK-KFxi5Z)